In [ ]:
#
# Simple example of a Julia linear pogramming model
#
# Convert from notebook to julia program:
#   python3 nb_to_jl.py lp_claude_revised_simplex.ipynb
#
 
using LinearAlgebra
using SparseArrays
using Random
using ArgParse

const DEFAULT_USE_PRESOLVE = false 
const DEFAULT_USE_INTERIOR_POINT_METHOD = true
const DEFAULT_USE_SPARSE_MATRIX = false
const DEFAULT_VERBOSE = true

global options = Dict()

const COMMAND_LINE_OPTION_FILENAME = "filename"
const COMMAND_LINE_OPTION_USE_PRESOLVE = "presolve"
const COMMAND_LINE_OPTION_USE_SPARSE_MATRIX = "sparse"
const COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD = "interior"
const COMMAND_LINE_OPTION_VERBOSE = "verbose"

const OPTION_FILENAME = "filename"
const OPTION_USE_PRESOLVE = "use_presolve"
const OPTION_USE_SPARSE_MATRIX = "use_sparse_matrix"
const OPTION_USE_INTERIOR_POINT_METHOD = "use_interior_point_method"
const OPTION_VERBOSE = "verbose"

# https://en.wikipedia.org/wiki/Revised_simplex_method
const MPS_EXAMPLE::String = """
NAME          APPLIED_INTEGER_PROGRAMMING_9_7

OBJSENSE
 MAX

ROWS
 N  OBJ
 L  ROW1
 L  ROW2
 L  ROW3

COLUMNS
    X1        OBJ       4
    X1        ROW1      1
    X1        ROW2      2
    X1        ROW3     -3
    X2        OBJ       3
    X2        ROW1      2
    X2        ROW2     -1
    X2        ROW3      2
    X3        OBJ       1
    X3        ROW1      3
    X3        ROW2      2
    X3        ROW3      1
    X4        OBJ       7
    X4        ROW1      1
    X4        ROW2      2
    X4        ROW3     -1
    X5        OBJ       6
    X5        ROW1     -3
    X5        ROW2      1
    X5        ROW3      2

RHS
    RHS1      ROW1      9
    RHS1      ROW2     10
    RHS1      ROW3     11

BOUNDS
 LO BOUND1    X1        0
 LO BOUND1    X2        0
 LO BOUND1    X3        0
 LO BOUND1    X4        0
 LO BOUND1    X5        0

ENDATA
"""
#=
/opt/homebrew/Cellar/highs/1.7.2/bin/highs --solution_file ex97_highs.txt ex97.mps
Running HiGHS 1.7.2 (git hash: n/a): Copyright (c) 2024 HiGHS under MIT licence terms
LP   ex97 has 3 rows; 5 cols; 15 nonzeros
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [1e+00, 7e+00]
  Bound  [0e+00, 0e+00]
  RHS    [9e+00, 1e+01]
Presolving model
3 rows, 5 cols, 15 nonzeros  0s
3 rows, 5 cols, 15 nonzeros  0s
Presolve : Reductions: rows 3(-0); columns 5(-0); elements 15(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.0999978563e+01 Ph1: 3(11); Du: 5(21) 0s
          4     9.4000000000e+01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  9.4000000000e+01
HiGHS run time      :          0.00

Model status
Optimal

# Primal solution values
Feasible
Objective 94
# Columns 5
X1 7
X2 10
X3 0
X4 0
X5 6
# Rows 3
ROW1 9
ROW2 10
ROW3 11

# Dual solution values
Feasible
# Columns 5
X1 0
X2 0
X3 -16.4285714285714
X4 -2.2380952380952
X5 0
# Rows 3
ROW1 1.4761904761905
ROW2 5.1904761904762
ROW3 2.6190476190476

# Basis
HiGHS v1
Valid
# Columns 5
1 1 0 0 1 
# Rows 3
2 2 2 
=#

In [ ]:
function is_running_in_notebook()
    # Check if running in VS Code's Jupyter notebook environment
    if (haskey(ENV, "VSCODE_PID") || haskey(ENV, "VSCODE_CWD"))
        return true
    # Check if running in a general Jupyter environment (including VS Code)
    elseif isdefined(Main, :IJulia) && Main.IJulia.inited
        return true
    else
        return false
    end
end

In [ ]:
function parse_commandline()
    s = ArgParseSettings()

    @add_arg_table s begin
        "--filename", "-f"
            help = "path to the problem file (mps format)"
            arg_type = String
            required = true
        "--interior", "-i"
            help = "use interior point method (LP only)"
            arg_type = Bool           
        "--min"
            help = "minimization of the objective function"
            action = :store_true            
        "--max"
            help = "maximization of the objective function"
            action = :store_true            
        "--presolve"
            help = "use presolve (default true)"
            arg_type = Bool          
        "--simplex"
            help = "use simplex method (default)"
            arg_type = Bool          
        "--sparse", "-s"
            help = "use sparce matrix representation"
            arg_type = Bool
        "--verbose", "-v"
            help = "verbose output"
            action = :store_true
    end

    return parse_args(s)
end

In [ ]:
# Define a struct to represent a Linear Programming problem
struct LPProblem
    is_minimize::Bool  # True if the objective is to minimize
    c::Vector{Float64}  # Objective function coefficients
    A::Matrix{Float64}  # Constraint matrix
    b::Vector{Float64}  # Right-hand side of constraints
    vars::Vector{String}  # Variable names
    constraint_types::Vector{Char}  # Constraint types
end

In [ ]:
function read_mps_from_string(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => Dict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name, row_name, value = words
            value = parse(Float64, value)
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = Dict()
            end
            sections["COLUMNS"][col_name][row_name] = value
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            bound_type, _, var_name, value = words
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = zeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    # Add bound constraints to A and b
    n_bound_constraints = count(x -> x > -Inf, lb) + count(x -> x < Inf, ub)
    A_with_bounds = zeros(n_constraints + n_bound_constraints, n_vars)
    b_with_bounds = zeros(n_constraints + n_bound_constraints)
    
    A_with_bounds[1:n_constraints, :] = A
    b_with_bounds[1:n_constraints] = b
    
    bound_constraint_index = n_constraints
    for i in 1:n_vars
        if lb[i] > -Inf
            bound_constraint_index += 1
            A_with_bounds[bound_constraint_index, i] = 1
            b_with_bounds[bound_constraint_index] = lb[i]
            push!(constraint_types, 'G')
        end
        if ub[i] < Inf
            bound_constraint_index += 1
            A_with_bounds[bound_constraint_index, i] = 1
            b_with_bounds[bound_constraint_index] = ub[i]
            push!(constraint_types, 'L')
        end
    end

    return LPProblem(is_minimize, c, A_with_bounds, b_with_bounds, vars, constraint_types)
end

In [ ]:
function presolve(lp::LPProblem; eps::Float64=1e-8)
    is_minimize, c, A, b = lp.is_minimize, lp.c, sparse(lp.A), lp.b
    vars, constraint_types = lp.vars, lp.constraint_types
    m, n = size(A)

    # Initialize masks for rows and columns to keep
    keep_rows = trues(m)
    keep_cols = trues(n)

    # Step 1: Remove zero columns
    for j in 1:n
        if nnz(A[:, j]) == 0 && abs(c[j]) < eps
            keep_cols[j] = false
        end
    end

    # Step 2: Remove zero rows
    for i in 1:m
        if nnz(A[i, :]) == 0
            if abs(b[i]) < eps
                keep_rows[i] = false
            else
                error("Infeasible problem: zero row with non-zero RHS")
            end
        end
    end

    # Step 3: Remove duplicate rows
    for i in 1:m
        if !keep_rows[i]
            continue
        end
        for j in (i+1):m
            if !keep_rows[j]
                continue
            end
            if A[i, :] ≈ A[j, :] && abs(b[i] - b[j]) < eps && constraint_types[i] == constraint_types[j]
                keep_rows[j] = false
            elseif A[i, :] ≈ -A[j, :] && abs(b[i] + b[j]) < eps && 
                   ((constraint_types[i] == '≤' && constraint_types[j] == '≥') || 
                    (constraint_types[i] == '≥' && constraint_types[j] == '≤'))
                keep_rows[j] = false
            end
        end
    end

    # Step 4: Fix variables and tighten bounds
    fixed_vars = Dict{String, Float64}()
    for j in 1:n
        col = A[:, j]
        if nnz(col) == 1
            i = findfirst(!iszero, col)
            if abs(col[i]) ≈ 1 && constraint_types[i] == '='
                val = b[i] / col[i]
                fixed_vars[vars[j]] = val
                keep_cols[j] = false
                b .-= val * col
            end
        end
    end

    # Apply the reductions
    A_new = A[keep_rows, keep_cols]
    b_new = b[keep_rows]
    c_new = c[keep_cols]
    vars_new = vars[keep_cols]
    constraint_types_new = constraint_types[keep_rows]

    # Adjust the objective for fixed variables
    obj_adjust = isempty(fixed_vars) ? 0.0 : sum(c[j] * val for (j, val) in enumerate(lp.vars) if haskey(fixed_vars, val))

    if options[OPTION_VERBOSE]
        println("\nPresolve summary:")
        println("  Original problem size: $(m) x $(n)")
        println("  Reduced problem size: $(sum(keep_rows)) x $(sum(keep_cols))")
        println("  Number of fixed variables: $(length(fixed_vars))")
        println("  Objective adjustment: $obj_adjust")
    end

    return LPProblem(is_minimize, c_new, Matrix(A_new), b_new, vars_new, constraint_types_new), fixed_vars, obj_adjust
end

In [ ]:
#=
Theoretical basis for each step:

Problem Formulation:

The Revised Simplex Method solves linear programming problems in the standard form:
Maximize c^T x
Subject to Ax = b
x ≥ 0


Slack Variables (Step 0):

Slack variables are added to convert inequality constraints to equality constraints.
This ensures that the problem is in standard form for the simplex method.


Basic and Non-Basic Variables:

The method partitions variables into basic (B) and non-basic (N) variables.
Initially, slack variables form the basis, corresponding to the identity matrix in the augmented constraint matrix.


Revised Simplex Iteration:
a. Basic Solution (Step 1):

Compute x_B = B^(-1) * b, where B is the basis matrix.
This gives the values of basic variables in the current solution.

b. Reduced Costs (Step 2):

Compute y = c_B' * B^(-1), where c_B are the objective coefficients of basic variables.
Calculate reduced costs: c_N - y' * A_N, where A_N are columns of non-basic variables.
Reduced costs measure the rate of change in the objective for unit increase in non-basic variables.

c. Optimality Check (Step 3):

If all reduced costs are non-positive, the current solution is optimal.
This is based on the theorem that a basic feasible solution is optimal if and only if all reduced costs are non-positive.

d. Entering Variable (Step 4):

Choose the non-basic variable with the most positive reduced cost to enter the basis.
This variable has the potential to improve the objective value the most.

e. Direction Computation (Step 5):

Compute d = B^(-1) * A_q, where A_q is the column of the entering variable.
This determines how basic variables change as the entering variable increases.

f. Unboundedness Check (Step 6):

If all elements of d are non-positive and the reduced cost is positive, the problem is unbounded.
This means we can increase the entering variable indefinitely, improving the objective without bound.

g. Leaving Variable (Step 7):

Perform the minimum ratio test: min(x_B_i / d_i) for d_i > 0.
This determines how far we can move along the edge without violating non-negativity constraints.

h. Basis Update (Step 8):

Swap the entering and leaving variables in the basis.
This moves to an adjacent extreme point of the feasible region.


Convergence:

The algorithm repeats these steps until optimality is reached or unboundedness is detected.
Each iteration either improves the objective value or determines that the problem is unbounded.
For non-degenerate problems, the algorithm is guaranteed to terminate in a finite number of steps.


Numerical Considerations:

Small tolerance (1e-10) is used for numerical stability in comparisons.
A maximum iteration limit prevents infinite loops in case of cycling (rare in practice, but possible in degenerate problems).

This implementation of the Revised Simplex Method efficiently solves linear programming problems by working with the inverse of the basis matrix and reduced costs, rather than maintaining the entire tableau as in the standard Simplex Method.
=#

function revised_simplex(lp::LPProblem)
    c, A, b = lp.c, lp.A, lp.b
    m, n = size(A)
    
    # If minimizing, negate the objective function to convert to a maximization problem
    if lp.is_minimize
        c = -c
    end
    
    println("\nInitial problem:")
    println("  Objective function coefficients c: ", lp.c)
    println("  Constraint matrix A: ", lp.A)
    println("  Right-hand side b: ", lp.b)
    println("  Variables: ", lp.vars) 
    println("  Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    
    # Step 0: Add slack variables to convert inequalities to equalities
    A = [A I(m)]  # Augment A with identity matrix for slack variables
    c = [c; zeros(m)]  # Extend objective coefficients with zeros for slack variables
    n = n + m  # Update number of variables
    
    # Initialize basis with slack variables
    B = collect(n-m+1:n)  # Indices of basic variables
    N = collect(1:n-m)    # Indices of non-basic variables
    
    println("  Initial basis: ", B)
    println("  Initial non-basic variables: ", N)
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]  # Extract basis matrix
        x_B = B_matrix \ b  # Solve B * x_B = b for basic variables
        
        println("  Basic solution: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (c[B]' / B_matrix)'  # Compute dual variables: y' * B = c_B'
        c_N = c[N] - A[:, N]' * y  # Compute reduced costs for non-basic variables
        
        println("  Reduced costs: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .<= 1e-10)  # If all reduced costs are non-positive, solution is optimal
            x = zeros(n)
            x[B] = x_B
            println("  Optimal solution found")
            obj_value = dot(c[1:length(lp.vars)], x[1:length(lp.vars)])
            if lp.is_minimize
                obj_value = -obj_value  # Convert back to minimization objective
            end
            return x[1:length(lp.vars)], obj_value
        end
        
        # Step 4: Choose entering variable (most positive reduced cost)
        e = argmax(c_N)
        q = N[e]
        
        println("  Entering variable: ", q)
        
        # Step 5: Compute direction of edge to traverse
        d = B_matrix \ A[:, q]
        
        println("  Direction: ", d)
        
        # Step 6: Check unboundedness
        if all(d .<= 1e-10)  # If direction is non-positive, problem is unbounded
            error("  Problem is unbounded")
        end
        
        # Step 7: Choose leaving variable (minimum ratio test)
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf  # Avoid division by zero or negative values
        l = argmin(filter(x -> x > 0, ratios))
        
        p = B[l]
        println("  Leaving variable: ", p)
        
        # Step 8: Update basis
        B[l] = q
        N[e] = p
        
        println("  New basis: ", B)
        println("  New non-basic variables: ", N)
        
        # Safeguard against infinite loops
        if iteration > 100
            println("  Maximum iterations reached")
            break
        end
    end
    
    error("Algorithm did not converge")
end

In [ ]:
function revised_simplex_sparse(lp::LPProblem)
    c, A, b = lp.c, sparse(lp.A), lp.b
    m, n = size(A)
    
    # If minimizing, negate the objective function to convert to a maximization problem
    if lp.is_minimize
        c = -c
    end
    
    println("\nInitial problem:")
    println("  Objective function coefficients c: ", lp.c)
    println("  Constraint matrix A: ", lp.A)
    println("  Right-hand side b: ", lp.b)
    println("  Variables: ", lp.vars)
    println("  Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    
    # Step 0: Add slack variables to convert inequalities to equalities
    A = [A sparse(I, m, m)]  # Augment A with sparse identity matrix for slack variables
    c = [c; zeros(m)]  # Extend objective coefficients with zeros for slack variables
    n = n + m  # Update number of variables
    
    # Initialize basis with slack variables
    B = collect(n-m+1:n)  # Indices of basic variables
    N = collect(1:n-m)    # Indices of non-basic variables
    
    println("  Initial basis: ", B)
    println("  Initial non-basic variables: ", N)
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]  # Extract basis matrix
        x_B = B_matrix \ Vector(b)  # Solve B * x_B = b for basic variables
        
        println("  Basic solution: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (Vector(c[B])' / B_matrix)'  # Compute dual variables: y' * B = c_B'
        c_N = c[N] - A[:, N]' * y  # Compute reduced costs for non-basic variables
        
        println("  Reduced costs: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .<= 1e-10)  # If all reduced costs are non-positive, solution is optimal
            x = spzeros(n)
            x[B] = x_B
            println("  Optimal solution found")
            obj_value = dot(c[1:length(lp.vars)], x[1:length(lp.vars)])
            if lp.is_minimize
                obj_value = -obj_value  # Convert back to minimization objective
            end
            return Array(x[1:length(lp.vars)]), obj_value
        end
        
        # Step 4: Choose entering variable (most positive reduced cost)
        e = argmax(c_N)
        q = N[e]
        
        println("  Entering variable: ", q)
        
        # Step 5: Compute direction of edge to traverse
        d = B_matrix \ Vector(A[:, q])  # Convert sparse column to dense vector
        
        println("  Direction: ", d)
        
        # Step 6: Check unboundedness
        if all(d .<= 1e-10)  # If direction is non-positive, problem is unbounded
            error("  Problem is unbounded")
        end
        
        # Step 7: Choose leaving variable (minimum ratio test)
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf  # Avoid division by zero or negative values
        l = argmin(filter(x -> x > 0, ratios))
        
        p = B[l]
        println("  Leaving variable: ", p)
        
        # Step 8: Update basis
        B[l] = q
        N[e] = p
        
        println("  New basis: ", B)
        println("  New non-basic variables: ", N)
        
        # Safeguard against infinite loops
        if iteration > 100
            println("  Maximum iterations reached")
            break
        end
    end
    
    error("Algorithm did not converge")
end

In [ ]:
function corrector_direction(A, r_b, r_s, x, s, Δx_aff, Δy_aff)
    println("Computing corrector direction:")
    
    # Get dimensions
    m, n = size(A)
    println("  m: $m, n: $n")

    # Dimension checks
    if size(A, 1) != m || size(A, 2) != n
        error("A should be an m×n matrix. Got size(A) = $(size(A))")
    end
    if length(r_b) != m || length(r_s) != m || length(s) != m || length(Δy_aff) != m
        error("r_b, r_s, s, and Δy_aff should be m-dimensional vectors. 
               Got lengths: r_b ($(length(r_b))), r_s ($(length(r_s))), 
               s ($(length(s))), Δy_aff ($(length(Δy_aff)))")
    end
    if length(x) != n || length(Δx_aff) != n
        error("x and Δx_aff should be n-dimensional vectors. 
               Got lengths: x ($(length(x))), Δx_aff ($(length(Δx_aff)))")
    end

    # Ensure vectors are column vectors
    r_b = vec(r_b)
    r_s = vec(r_s)
    x = vec(x)
    s = vec(s)
    Δx_aff = vec(Δx_aff)
    Δy_aff = vec(Δy_aff)

    println("  A: $A")
    println("  r_b: $r_b")
    println("  r_s: $r_s")
    println("  x: $x")
    println("  s: $s")
    println("  Δx_aff: $Δx_aff")
    println("  Δy_aff: $Δy_aff")

    # Compute Δs_aff
    Δs_aff = r_s - A * Δx_aff

    # Compute μ (barrier parameter)
    μ = (dot(x, A' * s) / n) * ((s + Δs_aff)' * (A * (x + Δx_aff)) / (s' * (A * x)))^3

    # Compute K matrix
    K = [spdiagm(0 => s) A; A' -spdiagm(0 => x)]
    println("  K: $K")

    # Compute rhs vector
    rhs = [r_b - A * Δx_aff;
           A' * r_s - A' * Δs_aff - (x .* (A' * Δs_aff) + A' * s .* Δx_aff - μ * ones(n))]
    println("  rhs: $rhs")

    # Solve system
    direction = K \ rhs
    println("  direction: $direction")

    # Extract and return the components of the direction
    Δy_cor = direction[1:m]
    Δx_cor = direction[m+1:end]

    return Δx_cor, Δy_cor
end

In [ ]:
function affine_scaling_direction(A, r_b, r_s, x, s, y, r_c)
    println("Computing affine scaling direction:")
    println("  A: $A")
    println("  r_b: $r_b")
    println("  r_s: $r_s")
    println("  x: $x")
    println("  s: $s")

    m, n = size(A)
    println("  m: $m, n: $n")

    # Ensure vectors are column vectors
    r_b = vec(r_b)
    r_s = vec(r_s)
    x = vec(x)
    s = vec(s)
    y = vec(y)
    r_c = vec(r_c)

    # Compute K matrix
    K = [spdiagm(0 => s) A; A' -spdiagm(0 => x)]
    println("  K: $K")

    # Compute rhs vector
    rhs = [r_b; -(A' * y + r_c)]
    println("  rhs: $rhs")

    # Solve system
    direction = K \ rhs
    println("  direction: $direction")

    # Extract and return the components of the direction
    Δy = direction[1:m]
    Δx = direction[m+1:end]

    return Δx, Δy
end

In [ ]:
function convert_inequality_constraints(A, b, constraint_types)
    m, n = size(A)
    println("Original A: $A")
    println("Original b: $b")
    println("Original constraint_types: $constraint_types")

    new_A = Matrix{Float64}(undef, 0, n)  # Initialize with correct number of columns
    new_b = Float64[]  # Initialize as empty vector
    new_constraint_types = Char[]

    for i = 1:m
        println("Processing constraint $i:")
        println("  A[i, :]: $(A[i, :])")
        println("  b[i]: $(b[i])")
        println("  constraint_types[i]: $(constraint_types[i])")

        if constraint_types[i] == 'L'
            println("  Constraint type: Less than or equal to")
            new_A = vcat(new_A, A[i, :]')  # Transpose A[i, :] to make it a row vector
            push!(new_b, b[i])
            push!(new_constraint_types, 'E')
        elseif constraint_types[i] == 'G'
            println("  Constraint type: Greater than or equal to")
            new_A = vcat(new_A, -A[i, :]')  # Transpose -A[i, :] to make it a row vector
            push!(new_b, -b[i])
            push!(new_constraint_types, 'E')
        elseif constraint_types[i] == 'E'
            println("  Constraint type: Equal to")
            new_A = vcat(new_A, A[i, :]')  # Transpose A[i, :] to make it a row vector
            push!(new_b, b[i])
            push!(new_constraint_types, 'E')
        end
    end

    println("Converted A: $new_A")
    println("Converted b: $new_b")
    println("Converted constraint_types: $new_constraint_types")

    return new_A, new_b, new_constraint_types
end

In [ ]:
function compute_step_length(v, Δv)
    α = 1.0
    for i in eachindex(v)
        if Δv[i] < 0
            α = min(α, -v[i] / Δv[i])
        end
    end
    return α
end

In [ ]:
#=
The rest of the Interior Point Method implementation looks correct. The main loop in the interior_point_method function follows the standard steps:

Compute residuals
Compute affine scaling direction
Compute corrector direction
Combine directions and update variables
Update barrier parameter
Check for convergence

The method also includes proper logging and debugging output, which is helpful for understanding the algorithm's progress.
To further improve the code, you might consider:

Adding error handling for cases where the problem doesn't converge within the maximum number of iterations.
Implementing a dynamic update strategy for the barrier parameter μ.
Adding more sophisticated step length calculations to ensure variables remain positive.
=#


function interior_point_method(lp::LPProblem; 
                               tolerance=1e-8, 
                               max_iterations=1000, 
                               barrier_parameter=0.1)
    # Initialize variables
    n = length(lp.vars)
    m = size(lp.A, 1)
    x = ones(n)
    s = ones(m)
    y = zeros(m)
    μ = barrier_parameter
    iteration = 0

    println("Starting IPM solver with parameters:")
    println("  Tolerance: $tolerance")
    println("  Max iterations: $max_iterations")
    println("  Barrier parameter: $barrier_parameter")

    # Convert inequality constraints to standard form
    A, b, constraint_types = convert_inequality_constraints(lp.A, lp.b, lp.constraint_types)
    println("Converted inequality constraints to standard form:")
    println("  A: $A")
    println("  b: $b")
    println("  constraint_types: $constraint_types")

    # Main loop
    while iteration < max_iterations
        println("Iteration $iteration:")

        # Compute residuals
        r_c = lp.c - A' * y
        r_b = b - A * x
        r_s = s - (A * x - b)
        println("  Residuals:")
        println("    r_c: $r_c")
        println("    r_b: $r_b")
        println("    r_s: $r_s")

        # Compute affine scaling direction
        Δx_aff, Δy_aff = affine_scaling_direction(A, r_b, r_s, x, s, y, r_c)
        Δs_aff = r_s - A * Δx_aff
        println("  Affine scaling direction:")
        println("    Δx_aff: $Δx_aff")
        println("    Δs_aff: $Δs_aff")
        println("    Δy_aff: $Δy_aff")

        # Compute corrector direction
        Δx_cor, Δy_cor = corrector_direction(A, r_b, r_s, x, s, Δx_aff, Δy_aff)
        Δs_cor = r_s - A * Δx_cor
        println("  Corrector direction:")
        println("    Δx_cor: $Δx_cor")
        println("    Δs_cor: $Δs_cor")
        println("    Δy_cor: $Δy_cor")

        # Combine directions and update variables
        Δx = Δx_aff + Δx_cor
        Δs = Δs_aff + Δs_cor
        Δy = Δy_aff + Δy_cor

        # Compute step length
        α_pri = compute_step_length(x, Δx)
        α_dual = compute_step_length(s, Δs)
        α = min(0.99 * min(α_pri, α_dual), 1)

        x += α * Δx
        s += α * Δs
        y += α * Δy
        println("  Updated variables:")
        println("    x: $x")
        println("    s: $s")
        println("    y: $y")

        # Update barrier parameter
        μ = (x' * s) / n
        println("  Updated barrier parameter: $μ")

        # Check convergence
        if norm([r_c; r_b; r_s]) < tolerance && μ < tolerance
            println("Converged in $iteration iterations")
            break
        end

        iteration += 1
    end

    return x, y, iteration
end

In [ ]:
# main program

if is_running_in_notebook()
    println("Running as a notebook")    
else
    println("Running as a script")
end

options[OPTION_USE_PRESOLVE] = DEFAULT_USE_PRESOLVE
options[OPTION_USE_SPARSE_MATRIX] = DEFAULT_USE_SPARSE_MATRIX
options[OPTION_USE_INTERIOR_POINT_METHOD] = DEFAULT_USE_INTERIOR_POINT_METHOD
options[OPTION_VERBOSE] = DEFAULT_VERBOSE

mps_string = MPS_EXAMPLE
if !is_running_in_notebook()
    parsed_args = parse_commandline()

    # Process command-line name
    if haskey(parsed_args, COMMAND_LINE_OPTION_FILENAME) && !isnothing(parsed_args[COMMAND_LINE_OPTION_FILENAME])
        options[OPTION_FILENAME] = parsed_args[COMMAND_LINE_OPTION_FILENAME]
        if isfile(options[OPTION_FILENAME])
            mps_string = return open(read, options[OPTION_FILENAME])
        else            
            error("File not found or could not read: '$(options[OPTION_FILENAME])'")
        end
    end    

    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_PRESOLVE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_PRESOLVE])
        options[OPTION_USE_PRESOLVE] = parsed_args[COMMAND_LINE_OPTION_USE_PRESOLVE]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_SPARSE_MATRIX) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_SPARSE_MATRIX])
        options[OPTION_USE_SPARSE_MATRIX] = parsed_args[COMMAND_LINE_OPTION_USE_SPARSE_MATRIX]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD])
        options[OPTION_USE_INTERIOR_POINT_METHOD] = parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_VERBOSE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_VERBOSE])
        options[OPTION_VERBOSE] = true
    end
end

if options[OPTION_VERBOSE]
    println("Options:")
    println("  Use presolve: $(options[OPTION_USE_PRESOLVE])")
    println("  Use sparse matrix: $(options[OPTION_USE_SPARSE_MATRIX])")
    println("  Use interior point method: $(options[OPTION_USE_INTERIOR_POINT_METHOD])")
    println("  Verbose output: $(options[OPTION_VERBOSE])")
end

# Example usage
lp = read_mps_from_string(mps_string)

# Presolve the problem
if options["use_presolve"]
    lp, fixed_vars, obj_adjust = presolve(lp)

    println("\nFixed variables:")
    for (var, val) in fixed_vars
        println("$var = $val")
    end

    println("\nReduced problem:")
    println("Minimize: $(lp.is_minimize)")
    println("c = $(lp.c)")
    println("A = $(lp.A)")
    println("b = $(lp.b)")
    println("vars = $(lp.vars)")
    println("constraint_types = $(lp.constraint_types)")

end

# solve the LP problem

if options["use_interior_point_method"]
        println("\nSolving the reduced problem using the Interior Point Method with dense matrices")
        execution_time = @elapsed x, y, iterations = interior_point_method(lp) 
else
    if options["use_interior_point_method"]
        println("\nSolving the reduced problem using the Revised Simplex Method with sparse matrices")
        execution_time = @elapsed x, obj_value = revised_simplex_sparse(lp)
    else
        println("\nSolving the reduced problem using the Revised Simplex Method with dense matrices")
        execution_time = @elapsed x, obj_value = revised_simplex(lp)
    end
end
println("\nExecution time: $execution_time seconds")

# show the solution

println("\nFinal solution:")
for (var, val) in zip(lp.vars, x[1:length(lp.vars)])
    println("$var = $(round(val, digits=6))")
end
println("Objective value: $(round(obj_value, digits=6))")

# A = [3.0 1.0 -3.0 2.0 1.0;
#         2.0 2.0 1.0 -1.0 2.0;
#         1.0 -1.0 2.0 2.0 -3.0]
# b = [9.0, 10.0, 11.0]
# c = [-1.0, -7.0, -6.0, -3.0, -4.0]  # Negated for maximization
# vars = ["X3", "X4", "X5", "X2", "X1"]
# lp = LPProblem(false, c, A, b, vars, ['≤', '≤', '≤'])

# x, obj_value = highs_inspired_ipm(lp)

# println("\nSolution:")
# for (var, val) in zip(lp.vars, x)
#     println("$var = $val")
# end
# println("Objective value: ", -obj_value)  # Negated back for maximization